In [ ]:
%pip install ../../../

In [ ]:
import torch 

device = torch.device("cuda:7")

In [ ]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY
transform = T.Compose([T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
    transform=transform
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test",
    transform=transform
)
testset.initialize()

In [ ]:
from spuco.models import model_factory 
from torch.optim import SGD
from spuco.end2end import LFF

bias_model=model_factory("lenet", trainset[0][0].shape, len(classes)).to(device)
model=model_factory("lenet", trainset[0][0].shape, len(classes)).to(device)
lff = LFF(
    trainset=trainset,
    bias_model=bias_model,
    debias_model=model,
    batch_size=4,
    bias_optimizer=SGD(bias_model.parameters(), lr=1e-3, weight_decay=5e-4),
    debias_optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4),
    q=0.2,
    num_epochs=1,
    device=device,
    verbose=True
)
lff.train()

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=bias_model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy

In [ ]:
evaluator.evaluate_spurious_task()

In [ ]:
import torch
from torch import nn 

a = torch.tensor([[1., 2.], [2., 4.]])
torch.softmax(a, dim=0)